In [2]:
from nanodo.model_factory import *
from nanodo.data import *
from nanodo.configs.default import *
from nanodo.train import *
import numpy as np
import orbax.checkpoint as ocp
from orbax.checkpoint import PyTreeCheckpointer
from flax.core import unfreeze
import jax.numpy as jnp
from nanodo.model import *

/home/allanz/miniconda3/envs/nanodo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 00:29:39.858580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742887779.876114 2360883 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742887779.881837 2360883 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742887779.897364 2360883 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more tha

In [16]:
c = get_config()
#checkpoint = "/home/allanz/nanodo_workdir/92000.orbax-checkpoint-tmp-138"
checkpoint = "/home/allanz/nanodo_workdir/90000/state"
params= PyTreeCheckpointer().restore(checkpoint)
test = unfreeze(params['params'])
params = params['params']

/home/allanz/miniconda3/envs/nanodo/lib/python3.11/site-packages/orbax/checkpoint/_src/serialization/type_handlers.py:1250: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [17]:
test_config = ml_collections.config_dict.create(
      D=512,  # model/embed dim  = qkv dim
      F=2048,  # FF inner dimension
      H=8,  # num attention heads
      L=128,  # max context/sequence length (move out of config?)
      N=6,  # number of transformer block layers
      dtype="float32",  # computation dtype.
      fsdp_enabled=True,  # True to shard the model.
      remat=False,  # Transformer block gradient checkpointing to save memory.
  )

In [30]:
tokenizer = get_py_tokenizer("tests/testdata/sentencepiece_cc_all.32000.100extra-sentencepiece.model")
vocab_size = tokenizer.GetPieceSize()
cfg = DoConfig(**test_config, V=vocab_size)  # pytype:disable=attribute-error
# model without fsdp
module = TransformerDo(cfg) 
print(module)
# model with fsdp
transformer, _ = get_model_and_loss(c, vocab_size)
print(transformer)
# same model but with dtype as jax.numpy.float32
docfg = model.DoConfig(D=512, H=8, L=128, N=6, V=vocab_size, F=2048)
m = model.TransformerDo(docfg)
print(m)


TransformerDo(
    # attributes
    docfg = DoConfig(D=512, H=8, L=128, N=6, V=32101, F=2048, kernel_init=<function variance_scaling.<locals>.init at 0x7f718bfbce00>, embed_init=<function variance_scaling.<locals>.init at 0x7f718bfbcea0>, dtype='float32', fsdp_enabled=True, remat=False)
)
TransformerDo(
    # attributes
    docfg = DoConfig(D=512, H=8, L=128, N=6, V=32101, F=2048, kernel_init=<function variance_scaling.<locals>.init at 0x7f718bfbce00>, embed_init=<function variance_scaling.<locals>.init at 0x7f718bfbcea0>, dtype='bfloat16', fsdp_enabled=True, remat=False)
)
TransformerDo(
    # attributes
    docfg = DoConfig(D=512, H=8, L=128, N=6, V=32101, F=2048, kernel_init=<function variance_scaling.<locals>.init at 0x7f718bfbce00>, embed_init=<function variance_scaling.<locals>.init at 0x7f718bfbcea0>, dtype=<class 'jax.numpy.float32'>, fsdp_enabled=True, remat=False)
)


In [37]:
rng = jax.random.PRNGKey(42)
_, init_rng = jax.random.split(rng)
x = jnp.ones((8, 512), dtype= jnp.int32)
initial_variables = jax.jit(m.init)(init_rng, x)

In [22]:
test_set = py_batched_tfds(
          tfds_name="c4_10k",
          split="train",
          context_size=512,
          worker_count=0,
          vocab_path="tests/testdata/sentencepiece_cc_all.32000.100extra-sentencepiece.model",
          batch_size = 8
          )
batch = next(iter(test_set))

ERROR:absl:Applying deprecated PyGrain MapOperation. Please use the grain.python.MapTransform.


In [47]:
from flax.linen import Partitioned

def make_partitioned(array, names):
    partition_array = Partitioned(array, names = names, mesh = None)
    return partition_array

def convert_attn_blocks(params):
    blocks = ["blocks_0", "blocks_1", "blocks_2", "blocks_3", "blocks_4", "blocks_5"]
    switches = {"attn_out_proj": (None, None, 'data'), "key": ('data', None), "query":('data', None), "value":('data', None)}

    for block in blocks:
          for switch in switches: 
               #print(params[block]["CausalAttn_0"][switch]["kernel"])
               params[block]["CausalAttn_0"][switch]["kernel"] = make_partitioned(params[block]["CausalAttn_0"][switch]["kernel"]["value"], switches[switch])

def convert_Mlp(params):
    blocks = ["blocks_0", "blocks_1", "blocks_2", "blocks_3", "blocks_4", "blocks_5"]
    switches = {"Dense_0": ('data', None), "Dense_1": ('data', None)}

    for block in blocks:
          for switch in switches: 
               #print(params[block]["Mlp_0"][switch]["kernel"])
               params[block]["Mlp_0"][switch]["kernel"] = make_partitioned(params[block]["Mlp_0"][switch]["kernel"]["value"], switches[switch])


def convert_embed(params):
    params["embed"]["embedding"] = make_partitioned(params["embed"]["embedding"]["value"], (None, 'data'))

def convert_pos_embed(params):
    params["pos_embed"]["embedding"] = make_partitioned(params["pos_embed"]["embedding"]["value"], (None, 'data'))


In [24]:
convert_attn_blocks(params)
convert_Mlp(params)
convert_embed(params)
convert_pos_embed(params)

In [38]:
logits = transformer.apply(initial_variables, x)
print(logits.shape)
print(logits)

(8, 512, 32101)
[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 ...

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan n

In [48]:
logits = transformer.apply({"params":params}, x)
print(logits.shape)
print(logits)

(2, 512, 32101)
[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]]


In [ ]:
docfg = model.DoConfig(D=128, H=16, L=256, N=4, V=1024, F=4 * 4)
m = model.TransformerDo(docfg)
rng = jax.random.PRNGKey(42)
_, init_rng = jax.random.split(rng)
input_shape = (2, 256)
x = jnp.ones(input_shape, dtype=jnp.int32)
initial_variables_new = jax.jit(m.init)(init_rng, x)
metrics = metrics_lib.Average()

In [40]:
logits = m.apply(initial_variables_new, x)
print(logits.shape)
print(logits)

(2, 256, 1024)
[[[-0.7536661   1.053213   -0.9188787  ...  0.93903595  1.8903488
   -0.03408347]
  [-0.8577212   1.9001383  -0.05233513 ...  1.1050556   1.8848469
   -0.76812565]
  [-0.63099265  1.6759     -0.31756437 ...  1.3335733   1.5211788
   -0.34489128]
  ...
  [-0.35996583  1.1647023  -0.72296077 ...  1.7744749   0.7932828
    0.21230151]
  [-0.36727864  1.2750794  -0.62616616 ...  1.8489721   0.8271866
    0.17445332]
  [-0.19379218  1.2730883  -0.5247805  ...  1.7327049   0.7116135
    0.30874372]]

 [[-0.7536661   1.053213   -0.9188787  ...  0.93903595  1.8903488
   -0.03408347]
  [-0.8577212   1.9001383  -0.05233513 ...  1.1050556   1.8848469
   -0.76812565]
  [-0.63099265  1.6759     -0.31756437 ...  1.3335733   1.5211788
   -0.34489128]
  ...
  [-0.35996583  1.1647023  -0.72296077 ...  1.7744749   0.7932828
    0.21230151]
  [-0.36727864  1.2750794  -0.62616616 ...  1.8489721   0.8271866
    0.17445332]
  [-0.19379218  1.2730883  -0.5247805  ...  1.7327049   0.7116135
   